In [ ]:
texts = [
    "Patient has a fever and headache",
    "X-ray shows a fracture in the left leg",
    "Prescribed 500mg paracetamol twice daily",
    "Blood test indicates high sugar levels",
    "MRI reveals a tumor in the brain",
    "Take two tablets of aspirin every morning",
    "Surgery scheduled for next Monday",
    "Blood pressure is elevated",
    "Symptoms include cough and fatigue",
    "CT scan confirms internal bleeding"
]

labels = [
    "symptom", "test", "treatment", "test", "test",
    "treatment", "treatment", "symptom", "symptom", "test"
]


In [ ]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Tokenize text
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, padding='post', maxlen=20)


In [ ]:
#Build the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=20),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
#Train the model
model.fit(padded, np.array(encoded_labels), epochs=200)


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4000 - loss: 1.0695
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4000 - loss: 1.0682
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4000 - loss: 1.0667
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4000 - loss: 1.0653
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4000 - loss: 1.0639
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4000 - loss: 1.0624
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4000 - loss: 1.0609
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4000 - loss: 1.0594
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4000 - loss: 1.0580
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4000 - loss: 1.0566
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4000 - loss: 1.0550
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4000 - lo

In [ ]:
model.save("medical_text_model.h5")


In [ ]:
#Use for testing
def classify_text(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=20, padding='post')
    pred = model.predict(pad)[0]
    label_idx = np.argmax(pred)
    label = label_encoder.inverse_transform([label_idx])[0]
    confidence = float(pred[label_idx])
    return label, confidence


In [ ]:
text = "ECG confirms herat condition"
label, conf = classify_text(text)
print(f"Prediction: {label}, Confidence: {conf:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Prediction: test, Confidence: 0.45
